In [1]:
import json
import sys, re

import requests

from copy import deepcopy
from pprint import pprint

In [2]:
"""let user pick any number of artists and respective songs"""
# artist0 = "nickelback"
# song0 = "far away"

artist0 = "justin bieber"
song0 = "baby"

# artist0 = 'the chainsmokers'
# song0 = 'something just like this'

# artist0 = input('pick artist? ').title()
# song0 = input(f'pick song for {artist0}? ')



print()
# artist1 = "drake"
# song1 = "god's plan"

# artist1 = 'taylor swift'
# song1 = 'look what you made me do'

artist1 = 'lady gaga'
song1 = 'just dance'

# artist1 = input('pick artist? ').title()
# song1 = input(f'pick song for {artist1}? ')

In [3]:
"""someone's function generating visuals"""  
def getsimsongs(artist, song):    # gives dict of sim songs
    try:
        urlraw = f'https://ws.audioscrobbler.com/2.0/?method=track.getsimilar&artist={artist}&track={song}&api_key=70894b101e887cf7fd9443bd5067f675&format=json'

        dicy = {}

        r = requests.get(urlraw)
        tmp = r.json()

        for i in range(len(tmp['similartracks']['track'])):
            songname = tmp['similartracks']['track'][i]['name']
            artistname = tmp['similartracks']['track'][i]['artist']['name']

            dicy[f'{artistname}|{songname}'] = 1
    except:
        print(f"""your artist ({artist.title()}) or song ({song.title()}) is WRONG""")
    else:
        return dicy


def match(dicy0, dicy1):   # gives matching dict of two dicts
    pot = {}
    
    for k0 in dicy0:
        for k1 in dicy1:
            if k0 == k1:
                pot[k1] = 1

    return pot


def addmore(dicy, n):    # adds new sim songs by specified pair
    pair = list(dicy)[n]
#     print(pair)
    
    artistpair = pair.split('|')[0]
    songpair = pair.split('|')[1]
    
    try:
        dicypair = getsimsongs(artistpair, songpair)
#         print(dicypair)

        dicynew = {**dicy, **dicypair}
    except:
        dicynew = dicy
        
#         print(f'FAILED searching... {pair}')
    
    return dicynew, pair


def iteration(dicy0, dicy1, hist0={}, hist1={}):  
    ##########################################################
    # check if sets inital match
    ##########################################################
#     print('data0', len(dicy0))
#     print('data1', len(dicy1))

    res = match(dicy0, dicy1)

    if len(res) >= 1:
#         print(len(res), res)
#         print(f'stop after add 0 pair(s)')

        print(f'DISCOVERED SONG(S) PLAYLIST: {len(res)}')

        for k in res:
            print(f"{k.split('|')[0]} - {k.split('|')[1]}")
    ##########################################################
    # expand song by song until match
    ##########################################################
    else:
        for i in range(max(len(dicy0), len(dicy1))):
            ind0 = 'n'
            ind1 = 'n'

            try:
                test0 = list(dicy0)[i]
            except:
                test0 = ''

            try:
                test1 = list(dicy1)[i]
            except:
                test1 = ''

            if test0.strip() and test0 not in hist0:
                dicy0, pair0 = addmore(dicy0, i)

#                 print('dicy0',len(dicy0))

                hist0[pair0] = 1

                ind0 = 'y'

            if test1.strip() and test1 not in hist1:
                dicy1, pair1 = addmore(dicy1, i)

#                 print('dicy1', len(dicy1))

                hist1[pair1] = 1

                ind1 = 'y'

            if not (ind0 == 'n' and ind1 == 'n'):
                res = match(dicy0, dicy1)

                if len(res) >= 1:
#                     print(len(res), res)
#                     print(f'stop after add {i + 1} pair(s)')

                    print(f'DISCOVERED SONG(S) PLAYLIST: {len(res)}')


                    for k in res:
                        print(f"{k.split('|')[0]} - {k.split('|')[1]}")

                    break

    return dicy0, dicy1, hist0, hist1

In [4]:
def main(artist0, song0, artist1, song1):
    d0 = getsimsongs(artist0, song0)
    d1 = getsimsongs(artist1, song1)

    # first iteration pair by pair
    try:
        d00, d11, h00, h11 = iteration(d0, d1)
    except:
        pass

In [5]:
main(artist0, song0, artist1, song1)

DISCOVERED SONG(S) PLAYLIST: 7
Selena Gomez & the Scene - Love You Like a Love Song
Nelly Furtado - Promiscuous
Rihanna - Only Girl (In the World)
Nicki Minaj - Starships
Little Mix - Shout Out To My Ex
Black Eyed Peas - I Gotta Feeling
Selena Gomez - Wolves
